In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter.simpledialog import askinteger
from tkinter import ttk
import random


class Minesweeper:
    def __init__(self, size, num_mines):
        self.size = size
        self.num_mines = num_mines
        self.board = [[' ' for _ in range(size)] for _ in range(size)]
        self.mine_positions = set()
        self.visible = [[' ' for _ in range(size)] for _ in range(size)]
        self.flags = [[' ' for _ in range(size)] for _ in range(size)]
        self.game_over = False
        self.initialize_board()


    def initialize_board(self):
        self.place_mines()
        self.calculate_numbers()


    def place_mines(self):
        while len(self.mine_positions) < self.num_mines:
            row = random.randint(0, self.size - 1)
            col = random.randint(0, self.size - 1)
            if (row, col) not in self.mine_positions:
                self.mine_positions.add((row, col))
                self.board[row][col] = '*'


    def calculate_numbers(self):
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == '*':
                    continue
                count = sum((row + i, col + j) in self.mine_positions
                            for i in range(-1, 2) for j in range(-1, 2)
                            if 0 <= row + i < self.size and 0 <= col + j < self.size)
                self.board[row][col] = str(count)


    def reveal(self, row, col):
        if self.flags[row][col] == 'F' or self.visible[row][col] != ' ':
            return
        if self.board[row][col] == '*':
            self.game_over = True
            return
        self._reveal_recursive(row, col)


    def _reveal_recursive(self, row, col):
        if self.visible[row][col] != ' ':
            return
        self.visible[row][col] = self.board[row][col]
        if self.board[row][col] == '0':
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if 0 <= row + i < self.size and 0 <= col + j < self.size:
                        self._reveal_recursive(row + i, col + j)


    def flag(self, row, col):
        if self.visible[row][col] == ' ':
            self.flags[row][col] = 'F' if self.flags[row][col] == ' ' else ' '


    def chord(self, row, col):
        if self.visible[row][col] in '12345678':
            adjacent_flags = sum(1 for i in range(-1, 2) for j in range(-1, 2)
                                 if 0 <= row + i < self.size and 0 <= col + j < self.size and self.flags[row + i][col + j] == 'F')
            if int(self.visible[row][col]) == adjacent_flags:
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if 0 <= row + i < self.size and 0 <= col + j < self.size and self.flags[row + i][col + j] == ' ':
                            self.reveal(row + i, col + j)
                            if self.game_over:
                                break




class MinesweeperGUI:
    def __init__(self, root, size, difficulty):
        self.root = root
        self.size = size
        self.difficulty = difficulty
        self.num_mines = self.calculate_mines(size, difficulty)
        self.game = Minesweeper(size, self.num_mines)
        self.buttons = [[None for _ in range(size)] for _ in range(size)]
        self.create_menu()
        self.create_widgets()
        self.update_buttons()


    def calculate_mines(self, size, difficulty):
        if difficulty == "Easy":
            return (size * size) // 10
        elif difficulty == "Medium":
            return (size * size) // 6
        elif difficulty == "Hard":
            return (size * size) // 4


    def create_menu(self):
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
       
        game_menu = tk.Menu(menubar, tearoff=0)
        game_menu.add_command(label="Restart", command=self.restart)
        game_menu.add_command(label="Change Size", command=self.change_size)
        game_menu.add_command(label="Change Difficulty", command=self.change_difficulty)
        menubar.add_cascade(label="Game", menu=game_menu)


    def create_widgets(self):
        self.frame = tk.Frame(self.root)
        self.frame.grid(row=0, column=0, sticky="nsew")


        for row in range(self.size):
            self.frame.grid_rowconfigure(row, weight=1)
            for col in range(self.size):
                self.frame.grid_columnconfigure(col, weight=1)
                button = tk.Button(self.frame, width=2, height=1,
                                   command=lambda r=row, c=col: self.on_button_click(r, c))
                button.bind('<Button-3>', lambda e, r=row, c=col: self.on_right_click(r, c))
                button.grid(row=row, column=col, sticky="nsew")
                self.buttons[row][col] = button


    def on_button_click(self, row, col):
        if self.game.game_over:
            return
        if self.game.visible[row][col] in '12345678':
            self.game.chord(row, col)
        else:
            self.game.reveal(row, col)
        self.update_buttons()
        if self.game.game_over:
            messagebox.showinfo("Game Over", "You clicked on a mine!")
            self.reveal_all()


    def on_right_click(self, row, col):
        if self.game.game_over:
            return
        self.game.flag(row, col)
        self.update_buttons()


    def update_buttons(self):
        for row in range(self.size):
            for col in range(self.size):
                if self.game.flags[row][col] == 'F':
                    self.buttons[row][col].config(text='F', fg='red')
                elif self.game.visible[row][col] != ' ':
                    self.buttons[row][col].config(text=self.game.visible[row][col], state="disabled")
                else:
                    self.buttons[row][col].config(text='')


    def reveal_all(self):
        for row in range(self.size):
            for col in range(self.size):
                if (row, col) in self.game.mine_positions:
                    self.buttons[row][col].config(text='*')
                else:
                    self.buttons[row][col].config(text=self.game.board[row][col])


    def restart(self):
        self.num_mines = self.calculate_mines(self.size, self.difficulty)
        self.game = Minesweeper(self.size, self.num_mines)
        self.update_widgets()
        self.update_buttons()


    def change_size(self):
        new_size = askinteger("Change Size", "Enter new size (between 5 and 20):", minvalue=5, maxvalue=20)
        if new_size:
            self.size = new_size
            self.restart()


    def change_difficulty(self):
        difficulty_window = tk.Toplevel(self.root)
        difficulty_window.title("Change Difficulty")
        tk.Label(difficulty_window, text="Select Difficulty:").pack(pady=10)
       
        difficulty_var = tk.StringVar(value=self.difficulty)
        combobox = ttk.Combobox(difficulty_window, textvariable=difficulty_var, values=["Easy", "Medium", "Hard"], state="readonly")
        combobox.pack(pady=10)


        def set_difficulty():
            self.difficulty = difficulty_var.get()
            self.restart()
            difficulty_window.destroy()


        tk.Button(difficulty_window, text="OK", command=set_difficulty).pack(pady=10)


    def update_widgets(self):
        for widget in self.frame.winfo_children():
            widget.destroy()
        self.buttons = [[None for _ in range(self.size)] for _ in range(self.size)]
        self.create_widgets()


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Minesweeper")
    root.rowconfigure(0, weight=1)
    root.columnconfigure(0, weight=1)
    size = 8
    difficulty = "Medium"
    game_gui = MinesweeperGUI(root, size, difficulty)
    root.mainloop()
